<a href="https://colab.research.google.com/github/eshaandinesh/Colab/blob/main/LinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files


uploaded = files.upload()

Saving CarPrice_Assignment.csv to CarPrice_Assignment.csv


In [ ]:
!pip install -q findspark
!pip install pyspark
import os
import findspark
import sys



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=66b319d13eec5146458e35b57d119a5414374bd5baede9cc9bdc39dad9ee95b7
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
#FIRST ATTEMPT WHICH DIDNT WORK BECAUSE OF FIT COMMAND

from pyspark.sql import SparkSession

from pyspark.ml.regression import LinearRegression

spark = SparkSession.builder \
    .appName("LinearRegression") \
    .getOrCreate()

df = spark.read.format("csv") \
    .option("header", "true") \
    .load("CarPrice_Assignment.csv")


lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

    # Fit the model
lrModel = lr.fit(df)

    # Print the coefficients and intercept for linear regression
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

    # Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)
    # $example off$

spark.stop()

IllegalArgumentException: ignored

In [ ]:
#USES SKLEARN


import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Load the data
df = pd.read_csv('CarPrice_Assignment.csv')

# Convert categorical variable into dummy/indicator variables.
df_encoded = pd.get_dummies(df)

# Assuming that 'price' is your target variable and the rest are features
X = df_encoded.drop('price', axis=1)
y = df_encoded['price']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the mean squared error of the predictions
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error: {mse}')


Mean Squared Error: 178507387.43178862


In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline

from collections import OrderedDict
import numpy as np


# Create a SparkSession
spark = SparkSession.builder.appName("LinearRegression").getOrCreate()

# Load the data
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("CarPrice_Assignment.csv")

# Assuming that 'price' is your target variable and the rest are features
feature_columns = df.columns  # list of column names excluding target
feature_columns.remove("price")  # replace "price" with your actual target column

# Convert categorical variable into dummy/indicator variables.
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df) for column in feature_columns]
df = Pipeline(stages=indexers).fit(df).transform(df)

# Assemble the features into a single vector
assembler = VectorAssembler(inputCols=[column+"_index" for column in feature_columns], outputCol="features")
df = assembler.transform(df)

# Split the data into training and test sets
train_data, test_data = df.randomSplit([0.8, 0.2])

# Create a linear regression model
lr = LinearRegression(featuresCol='features', labelCol='price')

# Fit the model to the training data
lr_model = lr.fit(train_data)

# Make predictions on the test set
predictions = lr_model.transform(test_data)

# Evaluate the model using RMSE (Root Mean Squared Error)
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="price", metricName="rmse")
rmse = evaluator.evaluate(predictions)

print(f'Root Mean Squared Error: {rmse}')

# Print the coefficients and intercept for linear regression
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

# Summarize the model over the training set and print out some metrics
trainingSummary = lr_model.summary
print("R-squared: %f" % trainingSummary.r2)

# Get the feature names from the VectorAssembler
feature_names = assembler.getInputCols()

# Get the coefficients from the LinearRegressionModel
coefficients = lr_model.coefficients


d = dict()
# Print the coefficients along with the feature names
for coef, feat in zip(coefficients, feature_names):
    #print(f'Feature: {feat}, Coefficient: {coef}')
    d[feat] = coef


d_sorted = dict(sorted(d.items(), key = lambda x: x[1], reverse = True))
#print(d_sorted)

for f in d_sorted:
    print(f,d_sorted[f])

AnalysisException: ignored